<a href="https://colab.research.google.com/github/824024445/Machine-learning-notes/blob/master/5%E6%94%AF%E6%8C%81%E5%90%91%E9%87%8F%E6%9C%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

也叫最大间隔分类



# 一、线性支持向量机分类
以鸢尾花数据为例  

硬间隔，软间隔：
软间隔就是允许一部分数据在边界内  
参数解释：
C：值越小间隔越“软”
loss:计算软间隔用的损失函数的类型

与逻辑回归分类器比较：
- 逻辑回归分类器是判断属于每个类别的概率，选择最大的那个
- 损失函数的判别项不一样。逻辑回归分类器每个项都有损失。而支持向量机有一个阈值，小于这个阈值则损失为0.

> 注：SVM为：Support Vector Machines支持向量机。  而SVC是Support Vector Classification支持向量分类

In [3]:
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler # 特征缩放。特征缩放后的边界会更好
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

iris = datasets.load_iris()
X = iris["data"][:, (2,3)] # 只选取长度和宽度两列
y = (iris["target"]==2).astype(np.int) # 只判断目标是否为Virginica这种花

svm_clf = Pipeline([
  ("scaler",StandardScaler()),
  ("linear_svc", LinearSVC(C=1, loss="hinge"))
])

svm_clf.fit(X, y)


Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('linear_svc',
                 LinearSVC(C=1, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='hinge', max_iter=1000, multi_class='ovr',
                           penalty='l2', random_state=None, tol=0.0001,
                           verbose=0))],
         verbose=False)

In [4]:
svm_clf.predict([[5.5, 1.7]])

array([1])

# 二、非线性支持向量机分类

## 2.1 方式1：转换数据集添加多项式特征

有些数据不是线性的，所以[**同多项式回归**](https://github.com/824024445/Machine-learning-notes/blob/master/4%E8%AE%AD%E7%BB%83%E6%A8%A1%E5%9E%8B.ipynb)。要对数据集通过PolyolynomialFeaturenomialFeature进行转换，增加特征

In [5]:
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

polynomial_svm_clf = Pipeline((
    ("poly_features", PolynomialFeatures(degree=3)),
    ("scaler", StandardScaler()),
    ("svm_clf", LinearSVC(C=10, loss="hinge"))
))

polynomial_svm_clf.fit(X, y)

Pipeline(memory=None,
         steps=[('poly_features',
                 PolynomialFeatures(degree=3, include_bias=True,
                                    interaction_only=False, order='C')),
                ('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svm_clf',
                 LinearSVC(C=10, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='hinge', max_iter=1000, multi_class='ovr',
                           penalty='l2', random_state=None, tol=0.0001,
                           verbose=0))],
         verbose=False)

## 2.2 方式2：核函数

### 2.2.1 核函数概要

> 具体解析见《机器学习》（西瓜书）--周志华 6.3节 核函数 

在数据并不是线性可分的情况下。可以将样本从原始空间映射到一个更高维的特征空间，使样本在新的特征空间线性可分。  
核函数即为将原样本映射为新样本的函数。  


---


常用核函数：
![替代文字](https://raw.githubusercontent.com/824024445/Machine-learning-notes/master/img/5-1.jpg)


---


常用的核函数的适用情况：
- 线性核:主要用于线性可分的情况。我们可以看到特征空间到输入空间的维度是一样的，其参数少|速度快。对于线性可分数据，其分类效果很理想，因此我们通常首先尝试用线性核函数来做分类，看看效果如何，如果不行再换别的。
- 多项式核：多项式核函数可以实现将低维的输入空间映射到高纬的特征空间，但是多项式核函数的参数多，当多项式的阶数比较高的时候，核矩阵的元素值将趋于无穷大或者无穷小，计算复杂度会大到无法计算。
- 高斯核：高斯径向基函数是一种局部性强的核函数，其可以将一个样本映射到一个更高维的空间内，该核函数是应用最广的一个，无论大样本还是小样本都有比较好的性能，而且其相对于多项式核函数参数要少，因此大多数情况下在不知道用什么核函数的时候，优先使用高斯核函数。
- 拉普拉斯核
- Sigmoid核：采用sigmoid核函数，支持向量机实现的就是一种多层神经网络。



---


**实践中核函数的选取依据：**  
因此，在选用核函数的时候，如果我们对我们的数据有一定的先验知识，就利用先验来选择符合数据分布的核函数；如果不知道的话，通常使用交叉验证1的方法，来试用不同的核函数，误差最小的即为效果最好的核函数，或者也可以将多个核函数结合起来，形成混合核函数。在吴恩达的课上，也曾经给出过一系列的选择核函数的方法：
- 如果特征的数量大到和样本数量差不多，则选用LR或者线性核的SVM；
- 如果特征的数量小，样本的数量正常，则选用SVM+高斯核函数；
- 如果特征的数量小，而样本的数量很大，则需要手工添加一些特征从而变成第一种情况。
- 注：其实最完美的就是高斯核函数，如果想要快，用线性。

> 本人想法：感觉如果将核方法设定为“poly”，那么就跟添加多项式特征的方法一样了。那么为什么又说性能比单纯的添加多项式特征好呢？应该是其它的几种核函数，poly核函数本身跟直接添加多项式特征一样。

### 2.2.2 多项式核
由于添加多项式特征在特征多时会产生大量的新特征，运行速度会变得很慢。  
可以理解成，不用往数据集中添加大量新特征，却能够得到同样好的结果，的一种数学方法。

参数解释：
- kernel：在算法中使用的内核类型。它必须是'linear'，'poly'，'rbf'，'sigmoid'，'precomputed'或者callable之一
- degree:多项式核函数的次数（'poly'）。被所有其他内核忽略。同PolynomialFeatures类，指定的是多项式的阶数

- coef0：核函数中的独立项。它只在'poly'和'sigmoid'中很重要。控制高阶多项式和低阶多项式对模型的影响
- C：错误术语的惩罚参数C，默认为1

In [6]:
from sklearn.svm import SVC

poly_kernel_svm_clf = Pipeline((
 # 不用使用PolynomialFeatures给数据集添加新特征了
 ("scaler", StandardScaler()),
 ("svm_clf", SVC(kernel="poly", degree=3, coef0=1, C=5))
 ))
poly_kernel_svm_clf.fit(X, y)

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svm_clf',
                 SVC(C=5, cache_size=200, class_weight=None, coef0=1,
                     decision_function_shape='ovr', degree=3,
                     gamma='auto_deprecated', kernel='poly', max_iter=-1,
                     probability=False, random_state=None, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

### 2.2.3 高斯核（rbf核）(默认项）
使用相似函数（高斯径向基函数），增加相似特征。  
它适用于特征少，数据量正常的情况下，因此是SVC类该项参数的默认值

参数：
gamma：值越大，拟合程度越高
C：同样，值越大，拟合程度越高

In [7]:
rbf_kernel_svm_clf = Pipeline((
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="rbf", gamma=5, C=0.01))
))

rbf_kernel_svm_clf.fit(X, y)

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svm_clf',
                 SVC(C=0.01, cache_size=200, class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma=5,
                     kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

# 三、SVM回归

不同于SVM分类试图找到两个类别间尽可能大的“街道”，SVM回归试图让尽可能多的样本进入街道。

## 3.1 线性SVM回归

In [8]:
from sklearn.svm import LinearSVR

svm_reg = LinearSVR(epsilon=0.5)
svm_reg.fit(X, y)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVR(C=1.0, dual=True, epsilon=0.5, fit_intercept=True,
          intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
          random_state=None, tol=0.0001, verbose=0)

## 3.2 非线性SVM回归
同非线性SVM分类，使用核函数

In [9]:
from sklearn.svm import SVR

svm_poly_reg = SVR(kernel="poly", degree=2, C=100, epsilon=0.1)
svm_poly_reg.fit(X, y)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVR(C=100, cache_size=200, coef0=0.0, degree=2, epsilon=0.1,
    gamma='auto_deprecated', kernel='poly', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)